In [1]:
import numpy as np
import pandas as pd
import itertools
import time
from concurrent.futures import ProcessPoolExecutor
import random
from functools import partial

In [2]:
# generate the model parameters
class bayesian_network_posterior:
    def __init__(self, the_R, tau2_R, kap_R, lam_R, mu, num_state, num_action, n_time, beta, v2):
        self.the_R = the_R
        self.tau2_R = tau2_R
        self.kap_R = kap_R
        self.lam_R = lam_R
        self.num_nodes = (num_state + num_action) * n_time
        self.num_state = num_state
        self.num_action = num_action
        self.mu = mu
        self.beta = beta
        self.v2 = v2
        self.n_time = n_time

    def posterior_sample(self, size=1, useFixed = True):
        p_beta = np.zeros(shape=(self.num_nodes,self.num_nodes, size))
        p_v2 = np.zeros(shape=(self.num_nodes, size))
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if self.tau2_R[i, j] != 0:
                    p_beta[i, j, ] = np.random.normal(loc=self.the_R[i,j], scale=np.sqrt(self.tau2_R[i,j]), size=size)
            gamma_rate = self.lam_R[i] / 2
            p_v2[i,] = 1 / np.random.gamma(shape=self.kap_R[i] / 2, scale=1/gamma_rate, size=size)
        if useFixed:
            p_beta = self.beta
            p_v2 = self.v2
        return (p_beta, p_v2, self.mu)

class bayesian_network:
    def __init__(self, mu, beta, v2, num_action, num_state, n_time, normalized = True, sample_mean=None, sample_sd=None):
        self.n_time = n_time
        if normalized:
            self.sample_mean = sample_mean
            self.sample_sd = sample_sd
            self.normalized = True
        self.initial_state_full = np.array([0.05, 0.00, 0.00, 30.00, 5.00,0.7])
        if num_state == 4:
            self.initial_state_base = self.initial_state_full[[0, 1, 3, 4, 5]]
        if num_state == 5:
            self.initial_state_base = self.initial_state_full
        self.mu = mu
        self.v2 = v2
        self.beta = beta
        self.num_action = num_action
        self.num_state = num_state
        self.n_factor = num_action + num_state
        self.beta_state = np.zeros(shape=(n_time, num_state, num_state)) # s -> s
        self.beta_action = np.zeros(shape=(n_time, num_action, num_state)) # a -> s
        for i in range(n_time-1):
            self.beta_state[i,:,:] = beta[(self.n_factor * (i+1) - num_state):(self.n_factor * (i+1)), (self.n_factor * (i + 2) - num_state): (self.n_factor * (i + 2))]
            self.beta_action[i,:,:] = beta[(self.n_factor * i):(self.n_factor * i + self.num_action), (self.n_factor * (i + 2) - num_state):(self.n_factor * (i + 2))]
        self.mu_a = []
        for i in range(self.n_time):
            temp_list = []
            for j in range(self.num_action):
                temp_list.append(self.mu[i * self.n_factor + j])
            self.mu_a.append(temp_list)

    def initial_state_generator(self, scale=10):
        init_states = self.initial_state_base + np.abs(np.random.normal(0, np.array(self.initial_state_base)/scale + 0.01))
        init_states = init_states[[0,1,3,4,5]] # init_states[:-1] * init_states[-1]
        self.initial_state = (init_states - self.sample_mean[(self.n_factor - self.num_state):self.n_factor]) / self.sample_sd[(self.n_factor - self.num_state):self.n_factor]

    def rescale_action(self, action, t, scale_method = "standard"):
        if not self.normalized:
            return action
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action + self.sample_mean[(self.n_factor * t):(self.n_factor * t + self.num_action)]
        else:
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action

    def rescale_state(self, state, t, scale_method = "standard"):
        if not self.normalized:
            return state
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state + self.sample_mean[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))]
        else:
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state
        
beta_gibbs = pd.read_csv('data/beta_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
v2_gibbs = pd.read_csv('data/v2_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None,
                        dtype=np.float64)
the_R = pd.read_csv('data/the_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
tau2_R = pd.read_csv('data/tau2_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
kap_R = pd.read_csv('data/kap_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
lam_R = pd.read_csv('data/lam_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
sd = pd.read_csv('data/sd_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
bn_post = bayesian_network_posterior(the_R.to_numpy(), tau2_R.to_numpy(), kap_R.to_numpy(), lam_R.to_numpy(),
                                         mu.to_numpy().flatten(), 5, 1, 36, beta_gibbs, v2_gibbs)
# mua = np.array(mu.to_numpy().reshape(36, 6)[:, 0]).reshape(1, 36)
# mus = np.array(mu.to_numpy().reshape(36, 6)[:, 1:]).reshape(5, 36)

# 这个函数可以生成5个状态和5个状态之间的协方差矩阵，随机生成相关性矩阵，代表了H=36个周期
def simulate(H = 36, nums = 5, seed = 1):
    np.random.seed(seed)
    V_mat = np.zeros((H*nums, H*nums))
    mat = (2*np.random.rand((H-1)*nums, (H-1)*nums)-1)/5
    V_mat[nums*1:nums*H, nums*1:nums*H] = mat.dot(mat.T)
    return V_mat

def new_simulate(H = 36, nums = 5, seed = 1):
    np.random.seed(seed)
    V_mat = np.zeros((H*nums, H*nums))
    mat = (2*np.random.rand(H*nums, H*nums)-1)/5
    V_mat[nums*0:nums*H, nums*0:nums*H] = mat.dot(mat.T)
    return V_mat

In [3]:
np.random.seed(1)
nums = 5
H = 3
# 只需要生成一个样本，用来当作真实的参数
ssize1 = 1
#p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize1, useFixed=False)
#v_mat = simulate(H=H, seed = 4) # 这个可以得到一个随机的random factor的协方差矩阵,定义为多元正态分布，均值为0
p_beta = np.load('p_beta.npy')
p_v2 = np.load('p_v2.npy')
mu0 = np.load('mu0.npy')
v_mat = np.load('v_mat.npy')
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64) 
s1 = np.array([0.05,0,30,5,0.7])
theta = np.load('theta.npy')

# b_r 和 c_r是组成奖励函数的参数
b_r = np.repeat(-534.52, H)
c_r = np.zeros((5, H))
c_r[1, H-1] = 1.29


beta = p_beta[:,:,0]
v2 = p_v2[:,0]
bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
# mu_a mu_s beta_a beta_s theta_0事先就可以确定下来，不需要在过程中计算
mua = np.array(bn.mu_a).reshape(1, 36)
mus = np.array(bn.mu.reshape(36, 6)[:,1:]).reshape(5,36)
betas = np.zeros((5, 5, 36))
for i in range(36):
    betas[:,:,i] = bn.beta_state[i,:,:]
betaa = np.transpose(bn.beta_action.reshape(36,5))

# 为了减少维数，进行截断
mua = mua[:,:H]
mus = mus[:,:H]
betas = betas[:,:,:H]
betaa = betaa[:,:H]
theta = theta[:,:H-1]

theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)

#permutations = itertools.permutations([i for i in range(H*nums)])


In [4]:
# 定义一些基本的函数
def reward(b_r, c_r, a, s):
    return np.dot(b_r,a) + np.dot(c_r, s)

def policy(mua, mus, theta, s):
    return mua + np.dot(theta, s - mus)

# 这里的转移函数暂时不考虑随机因子
def transition(mus1, mus, mua, betas, betaa,s, a):
    return mus1 + np.dot(betas, s - mus) + betaa*(a - mua)

In [5]:
# 用于计算理论上真实的SV
def R2(tt,t):
    global betas,betaa,theta_0,nums
    R = np.eye(nums)
    while tt <= t:
        # betas 是5*5*36
        tmp = betas[:,:, tt] + np.outer(betaa[:,tt], theta_0[:,tt])
        R = np.dot(R,tmp)
        tt = tt+1
    return R

alpha = np.zeros((H,nums))
for t in range(H):
    alpha[t] = np.dot(b_r[t], theta_0[:,t]) + c_r[:,t]


def R1(t):
    global H,alpha
    R = 0
    for i in range(t,H):
        R += np.dot(alpha[i,:], R2(t, i-1))
    return R

# 接下来用一个列表填充关于R的内容
R = []
for t in range(H):
    R.append(R1(t))
R = np.concatenate(R) # 一共5*36个随机因子，组成对应180个R


# 接下来定义价值函数，注意180个随机因子的协方差矩阵是v_mat，假设来自于均值为0的正态
# 给定一个索引的集合，然后用来构建相对应的R和协方差矩阵
def value_function(subset):
    # 对这些随机变量索引的集合排序
    global H,nums,R,v_mat
    all_list = [i for i in range(H*nums)]
    u_list = sorted(subset)
    neg_u_list = [item for item in all_list if item not in u_list]
    R_u = []
    R_neg = []
    for i in range((H-1)*nums):
        if i in u_list:
            R_u.append(R[i])
        else:
            R_neg.append(R[i])
    R_u = np.array(R_u)
    R_neg = np.array(R_neg)
    # 下面是协方差
    length_u = len(R_u)
    length_neg = len(R_neg)
    sigma_u = np.zeros((length_u,length_u))
    sigma_neg_u = np.zeros((length_neg, length_u))
    # 先填写sigma u
    for i in range(length_u):
        for j in range(length_u):
            sigma_u[i][j] = v_mat[u_list[i]][u_list[j]]
    # 填写sigma -u u
    for i in range(length_neg):
        for j in range(length_u):
            sigma_neg_u[i][j] = v_mat[neg_u_list[i]][u_list[j]]
    if np.linalg.det(sigma_u) == 0: # 没有逆矩阵，使用伪逆
        return (R_neg @ sigma_neg_u @ np.linalg.pinv(sigma_u) + R_u)@sigma_u@(R_neg @ sigma_neg_u @ np.linalg.pinv(sigma_u) + R_u)
    else:  
        return (R_neg @ sigma_neg_u @ np.linalg.inv(sigma_u) + R_u)@sigma_u@(R_neg @ sigma_neg_u @ np.linalg.inv(sigma_u) + R_u)

def factorial_iterative(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

def get_elements_before(lst, element):
    # 获取目标元素在列表中的索引
    if element in lst:
        index = lst.index(element)
        # 返回索引之前的所有元素
        return lst[:index]
    else:
        return []  # 如果目标元素不在列表中，返回空列表

def shapley_value(i, subset):
    global H,nums
    res = 0
    subset_before = get_elements_before(subset, i)
    subset_include = subset_before + [i]
    res += value_function(subset_include) - value_function(subset_before)
    return res

In [8]:
s_time = time.time()
shapley = np.zeros(H*nums)
inputs = [i for i in range(H*nums)]
P = 50000
for _ in range(P):
    perm = [i for i in range(H*nums)]
    random.shuffle(perm)
    with ProcessPoolExecutor() as executor:
    # 使用partial将fixed_param绑定到shapley_value函数
        results = list(executor.map(partial(shapley_value, subset = perm), inputs))
    shapley += np.array(results)
shapley = shapley/P

e_time = time.time()
print('算法总运行时间: ', e_time-s_time)
print(shapley)

算法总运行时间:  2055.7565779685974
[-2.45449883e-15 -2.06000550e-15 -2.74553713e-15 -1.79397830e-15
 -2.30215846e-15  5.66438171e+02  7.28224239e+01  6.63255965e+02
  1.42026969e+02  1.93420153e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00]
